In [1]:
from tqdm import tqdm
import pandas as pd
from ergochemics import mapping
from typing import List
import multiprocessing
import os
multiprocessing.set_start_method("fork", force=True)
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [2]:
rxns_df_input_filepath = "../data/raw/enzymemap_v2_brenda2023.csv"
rxns_df_output_filepath = "../data/interim/enzymemap_v2_brenda2023_JN_mapped_unique_rxns.parquet"

# enable batching to map large numbers of reactions without running out of memory
use_batching = True
batch_size = 2500  # number of reactions to process in each batch
batch_num = 16  # current batch number
start_idx = batch_num * batch_size  # starting index for the current batch
end_idx = (batch_num + 1) * batch_size  # ending index for the current batch

In [ ]:
def make_rule_id(n: int, prefix: str = "rule", width: int = 4) -> str:
    """
    Convert an integer into a zero-padded rule ID of the form 'rule0001'.

    Args:
        n (int): The integer to convert.
        prefix (str): Optional prefix before the number. Defaults to "rule".
        width (int): Zero-padding width. Defaults to 4.

    Returns:
        str: Formatted rule ID (e.g., 'rule0004').
    """
    if n < 1:
        raise ValueError("Input must be >= 1.")
    return f"{prefix}{n:0{width}d}"

# extract and create a list of all minimal operators' SMARTS strings
gen_rxn_operators_df = pd.read_csv("../data/raw/JN1224MIN_rules.tsv", delimiter='\t')
gen_rxn_operators_list: List[str] = gen_rxn_operators_df["SMARTS"].to_list()

# extract and create a list of all unmapped reactions
enzymatic_rxns_df = pd.read_csv(rxns_df_input_filepath)
enzymatic_rxns_df = enzymatic_rxns_df[~enzymatic_rxns_df['mapped'].duplicated()]

# rewrite output filepath if batching is used
if use_batching:
    rxns_df_output_filepath = f"{rxns_df_output_filepath.replace('.parquet', f'_batch{batch_num}.parquet')}"
    enzymatic_rxns_df = enzymatic_rxns_df[start_idx:end_idx]

unmapped_rxns_list: List[str] = enzymatic_rxns_df["unmapped"].to_list()

# remove all hydrogen ions from rxn strings so that they can be mapped by Stefan's ergochemics
cleaned_rxns_list: List[str] = []

for rxn in unmapped_rxns_list:
    rxn = rxn.replace(".[H+]","").replace("[H+].","")
    cleaned_rxns_list.append(rxn)

def map_single_reaction(args):
    """(index, rxn, operator_list) → (index, mapped_ops)"""
    idx, rxn, gen_rxn_operators_list = args

    mapped_ops = []
    try:
        for i, operator in enumerate(gen_rxn_operators_list):
            try:
                mapped_rxn = mapping.operator_map_reaction(rxn=rxn, operator=operator)
                if mapped_rxn.did_map:
                    mapped_ops.append(make_rule_id(i+1))
            except Exception:
                pass
    except Exception as e:
        return idx, f"__WORKER_FAILED__: {repr(e)}"

    return idx, mapped_ops

# ---- MAIN ----
rxns = cleaned_rxns_list
tasks = [(i, rxn, gen_rxn_operators_list) for i, rxn in enumerate(rxns)]

results = [None] * len(tasks)

with multiprocessing.Pool(os.cpu_count()) as p:
    for idx, mapped_ops in tqdm(
        p.imap_unordered(map_single_reaction, tasks),
        total=len(tasks),
        desc="Mapping reactions",
    ):
        results[idx] = mapped_ops

all_mapped_operators = results

enzymatic_rxns_df["all_mapped_operators"] = all_mapped_operators

def get_top_operator(op_list):
    """
    Given a list like ['rule0002', 'rule0754'], return the one with
    the smallest integer value (e.g. 'rule0002').
    """
    if not op_list:
        return None  # or np.nan if you prefer

    # extract integer part: "rule0034" → 34
    nums = [int(op.replace("rule", "")) for op in op_list]

    # lowest rule number
    min_num = min(nums)

    # convert back to rule format
    return f"rule{min_num:04d}"

enzymatic_rxns_df["top_mapped_operator"] = (
    enzymatic_rxns_df["all_mapped_operators"]
    .apply(get_top_operator))

Mapping reactions:   5%|▌         | 135/2500 [00:58<00:47, 49.43it/s] 

Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1

Mapping reactions:   6%|▌         | 143/2500 [00:58<00:41, 56.71it/s]


Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc[nH]c(=O)[nH]1.O>>N.O=c1nc[nH]c(=O)[nH]1
Error parsing tautomerized reac

Mapping reactions:   6%|▌         | 149/2500 [00:58<00:56, 41.91it/s]

...

Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for: N=c1nc(Cl)[nH]c(=N)[nH]1.O>>N.N=c1nc(Cl)[nH]c(=O)[nH]1
Error parsing tautomerized reactant for:

Mapping reactions:  47%|████▋     | 1173/2500 [01:48<00:08, 159.58it/s]

Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCN
Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCN
Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCN
Error parsing tautomerized reactant for: CCN=c1nc(SC)[nH]c(=NC(C)C)[nH]1.O>>CCN.CSc1nc(=O)[nH]c(=NC(C)C)[nH]1
Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCN
Error parsing tautomerized reactant for: CCN=c1nc(SC)[nH]c(=NC(C)C)[nH]1.O>>CCN.CSc1nc(=O)[nH]c(=NC(C)C)[nH]1
Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCNError parsing tautomerized reactant for: CCN=c1nc(SC)[nH]c(=NC(C)C)[nH]1.O>>CCN.CSc1nc(=O)[nH]c(=NC(C)C)[nH]1

Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCNError parsi

Mapping reactions:  48%|████▊     | 1189/2500 [01:49<00:25, 52.20it/s] 


Error parsing tautomerized reactant for: CCN=c1nc(SC)[nH]c(=NC(C)C)[nH]1.O>>CCN.CSc1nc(=O)[nH]c(=NC(C)C)[nH]1Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCN


Error parsing tautomerized reactant for: CCN=c1nc(OC)[nH]c(=NC(C)C)[nH]1.O>>CCN.COc1nc(=O)[nH]c(=NC(C)C)[nH]1Error parsing tautomerized reactant for: CC(C)N=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N.CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCNError parsing tautomerized reactant for: CCN=c1nc(SC)[nH]c(=NC(C)C)[nH]1.O>>CCN.CSc1nc(=O)[nH]c(=NC(C)C)[nH]1



Error parsing tautomerized reactant for: CCN=c1nc(SC)[nH]c(=NC(C)C)[nH]1.O>>CCN.CSc1nc(=O)[nH]c(=NC(C)C)[nH]1Error parsing tautomerized reactant for: CCN=c1nc(Cl)[nH]c(=NC(C)C)[nH]1.O>>CC(C)N=c1[nH]c(Cl)nc(=O)[nH]1.CCNError parsing tautomerized reactant for: CCN=c1nc(OC)[nH]c(=NC(C)C)[nH]1.O>>CCN.COc1nc(=O)[nH]c(=NC(C)C)[nH]1

Err

In [ ]:
enzymatic_rxns_df.head(5)

,rxn_idx,mapped,unmapped,orig_rxn_text,rule,rule_id,source,steps,quality,natural,organism,protein_refs,protein_db,ec_num,all_mapped_operators,top_mapped_operator
263812,250955,[CH3:1][CH2:2][C@H:3]([CH3:4])[C@H:5]([NH:6][C...,CC[C@H](C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)CNC(=O)...,Tyr-Pro-Phe-Pro-Gly-Pro-Ile + H2O = Tyr-Pro-Ph...,[#6:1]-[#7:2].[#8:3]>>[#6:1]-[#8:3].[#7:2],280,direct,single,1.0,False,Cavia porcellus,[],NaN,3.4.24.16,"[rule0013, rule0056, rule0099, rule0122, rule0...",rule0013
263819,250960,[CH3:1][CH2:2][CH:3]([CH3:4])[C@H:5]([NH:6][C:...,CCC(C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@...,pGlu-Leu-Tyr-Glu-Asn-Lys-Pro-Arg-Arg-Pro-Tyr-I...,[#6:1]-[#7:2].[#8:3]>>[#6:1]-[#8:3].[#7:2],280,direct,single,1.0,False,Cavia porcellus,[],NaN,3.4.24.16,"[rule0013, rule0056, rule0099, rule0122, rule0...",rule0013
263832,250967,[CH3:1][S:2][CH2:3][CH2:4][C@H:5]([NH:6][C:7](...,CSCC[C@H](NC(=O)[C@H](CC(C)C)NC(=O)CNC(=O)[C@H...,Arg-Pro-Lys-Pro-Gln-Gln-Phe-Phe-Gly-Leu-Met-NH...,[#6:1]-[#7:2].[#8:3]>>[#6:1]-[#8:3].[#7:2],280,suggested,single,1.0,False,Cavia porcellus,[],NaN,3.4.24.16,"[rule0013, rule0056, rule0099, rule0122, rule0...",rule0013
263856,250990,[CH3:1][CH:2]([CH3:3])[CH2:4][C@H:5]([NH:6][C:...,CC(C)C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)CNC(=O...,Tyr-Gly-Gly-Phe-Leu-Arg + H2O = Tyr-Gly-Gly-Ph...,[#6:1]-[#7:2].[#8:3]>>[#6:1]-[#8:3].[#7:2],280,direct,single,1.0,False,Rattus norvegicus,[],NaN,3.4.24.16,"[rule0013, rule0056, rule0099, rule0122, rule0...",rule0013
263857,250991,[CH3:1][CH2:2][C@H:3]([CH3:4])[C@H:5]([NH:6][C...,CC[C@H](C)[C@H](NC(=O)[C@H](CCCNC(=N)N)NC(=O)[...,Tyr-Gly-Gly-Phe-Leu-Arg-Arg-Ile-Arg + H2O = Ty...,[#6:1]-[#7:2].[#8:3]>>[#6:1]-[#8:3].[#7:2],280,direct,single,1.0,False,Rattus norvegicus,[],NaN,3.4.24.16,"[rule0013, rule0056, rule0099, rule0122, rule0...",rule0013


In [ ]:
enzymatic_rxns_df.to_parquet(rxns_df_output_filepath, index=False)